## Ugrađivanja

U našem prethodnom primjeru radili smo s vektorima vreće riječi visoke dimenzionalnosti duljine `vocab_size`, te smo eksplicitno pretvarali niskodimenzionalne vektore pozicijskog prikaza u rijetke one-hot prikaze. Ovaj one-hot prikaz nije memorijski učinkovit. Osim toga, svaka se riječ tretira neovisno o drugima, pa one-hot kodirani vektori ne izražavaju semantičke sličnosti između riječi.

U ovoj jedinici nastavit ćemo istraživati **News AG** skup podataka. Za početak, učitajmo podatke i preuzmimo neke definicije iz prethodne jedinice.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Što je embedding?

Ideja **embeddinga** je predstavljati riječi pomoću gusto raspoređenih vektora niže dimenzionalnosti koji odražavaju semantičko značenje riječi. Kasnije ćemo raspraviti kako izgraditi smislene word embeddinge, ali za sada razmotrimo embeddinge kao način smanjenja dimenzionalnosti vektora riječi.

Dakle, embedding sloj uzima riječ kao ulaz i proizvodi izlazni vektor određene veličine `embedding_size`. Na neki način, vrlo je sličan `Dense` sloju, ali umjesto da uzima one-hot kodirani vektor kao ulaz, može uzeti broj riječi.

Korištenjem embedding sloja kao prvog sloja u našoj mreži, možemo prijeći s modela vreće riječi (bag-of-words) na model **embedding vreće** (embedding bag), gdje prvo pretvaramo svaku riječ u našem tekstu u odgovarajući embedding, a zatim izračunavamo neku agregacijsku funkciju nad svim tim embeddingima, poput `sum`, `average` ili `max`.

![Slika koja prikazuje embedding klasifikator za pet riječi u nizu.](../../../../../translated_images/hr/embedding-classifier-example.b77f021a7ee67eee.webp)

Naša neuronska mreža klasifikatora sastoji se od sljedećih slojeva:

* `TextVectorization` sloj, koji uzima string kao ulaz i proizvodi tenzor brojeva tokena. Odredit ćemo razumnu veličinu vokabulara `vocab_size` i zanemariti riječi koje se rjeđe koriste. Ulazni oblik bit će 1, a izlazni oblik bit će $n$, budući da ćemo dobiti $n$ tokena kao rezultat, pri čemu svaki od njih sadrži brojeve od 0 do `vocab_size`.
* `Embedding` sloj, koji uzima $n$ brojeva i smanjuje svaki broj na gusti vektor određene duljine (100 u našem primjeru). Tako će ulazni tenzor oblika $n$ biti transformiran u $n\times 100$ tenzor.
* Agregacijski sloj, koji uzima prosjek ovog tenzora duž prve osi, tj. izračunat će prosjek svih $n$ ulaznih tenzora koji odgovaraju različitim riječima. Za implementaciju ovog sloja koristit ćemo `Lambda` sloj i u njega proslijediti funkciju za izračunavanje prosjeka. Izlaz će imati oblik 100 i predstavljat će numeričku reprezentaciju cijelog ulaznog niza.
* Završni `Dense` linearni klasifikator.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


U ispisu `summary`, u stupcu **output shape**, prva dimenzija tenzora `None` odgovara veličini minibatcha, a druga odgovara duljini niza tokena. Svi nizovi tokena u minibatchu imaju različite duljine. O tome kako se nositi s tim razgovarat ćemo u sljedećem odjeljku.

Sada krenimo s treniranjem mreže:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Napomena** da gradimo vektorizator na temelju podskupa podataka. To se radi kako bi se ubrzao proces, i to može rezultirati situacijom u kojoj nisu svi tokeni iz našeg teksta prisutni u rječniku. U tom slučaju, ti tokeni će biti zanemareni, što može dovesti do neznatno manje točnosti. Međutim, u stvarnom životu podskup teksta često daje dobru procjenu rječnika.


### Rad s različitim veličinama sekvenci varijabli

Razumijmo kako se odvija treniranje u minibatchovima. U gornjem primjeru, ulazni tenzor ima dimenziju 1, a koristimo minibatcheve duljine 128, tako da je stvarna veličina tenzora $128 \times 1$. Međutim, broj tokena u svakoj rečenici je različit. Ako primijenimo sloj `TextVectorization` na jedan ulaz, broj vraćenih tokena bit će različit, ovisno o tome kako je tekst tokeniziran:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Međutim, kada primijenimo vektorizator na nekoliko sekvenci, mora proizvesti tenzor pravokutnog oblika, pa popunjava neiskorištene elemente s PAD tokenom (što je u našem slučaju nula):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Ovdje možemo vidjeti ugrađivanja:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Napomena**: Kako bi se smanjila količina popunjavanja, u nekim slučajevima ima smisla poredati sve sekvence u skupu podataka prema rastućoj duljini (ili, točnije, prema broju tokena). To će osigurati da svaki minibatch sadrži sekvence slične duljine.


## Semantičke ugradnje: Word2Vec

U našem prethodnom primjeru, sloj za ugradnju naučio je mapirati riječi u vektorske reprezentacije, no te reprezentacije nisu imale semantičko značenje. Bilo bi korisno naučiti vektorsku reprezentaciju na način da slične riječi ili sinonimi odgovaraju vektorima koji su blizu jedni drugima prema nekoj metričkoj udaljenosti (na primjer, euklidskoj udaljenosti).

Da bismo to postigli, potrebno je unaprijed istrenirati naš model za ugradnju na velikoj zbirci teksta koristeći tehniku poput [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Ova tehnika temelji se na dvije glavne arhitekture koje se koriste za stvaranje distribuirane reprezentacije riječi:

 - **Kontinuirana vreća riječi** (CBoW), gdje treniramo model da predvidi riječ iz okolnog konteksta. Za zadani ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, cilj modela je predvidjeti $W_0$ na temelju $(W_{-2},W_{-1},W_1,W_2)$.
 - **Kontinuirani skip-gram** je suprotan CBoW-u. Model koristi okolni prozor kontekstualnih riječi kako bi predvidio trenutnu riječ.

CBoW je brži, dok je skip-gram sporiji, ali bolje predstavlja riječi koje se rjeđe pojavljuju.

![Slika koja prikazuje algoritme CBoW i Skip-Gram za pretvaranje riječi u vektore.](../../../../../translated_images/hr/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Kako bismo eksperimentirali s Word2Vec ugradnjom unaprijed istreniranom na Google News skupu podataka, možemo koristiti biblioteku **gensim**. Ispod nalazimo riječi koje su najsličnije riječi 'neural'.

> **Napomena:** Kada prvi put kreirate vektore riječi, njihovo preuzimanje može potrajati!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Također možemo izdvojiti vektorsko urezivanje iz riječi, koje će se koristiti za treniranje modela klasifikacije. Urezivanje ima 300 komponenti, ali ovdje prikazujemo samo prvih 20 komponenti vektora radi jasnoće:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Sjajna stvar kod semantičkih ugradnji je da možete manipulirati vektorskim kodiranjem na temelju semantike. Na primjer, možemo tražiti riječ čija je vektorska reprezentacija što je moguće bliža riječima *kralj* i *žena*, a što je moguće udaljenija od riječi *muškarac*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Primjer iznad koristi neku internu GenSym magiju, ali osnovna logika je zapravo prilično jednostavna. Zanimljiva stvar kod ugradnji (embeddings) je da možete izvoditi normalne vektorske operacije na vektorskim ugradnjama, a to bi odražavalo operacije na značenjima riječi. Primjer iznad može se izraziti u terminima vektorskih operacija: izračunavamo vektor koji odgovara **KRALJ-MUŠKARAC+ŽENA** (operacije `+` i `-` izvode se na vektorskim prikazima odgovarajućih riječi), a zatim pronalazimo najbližu riječ u rječniku tom vektoru:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NAPOMENA**: Morali smo dodati male koeficijente na vektore *man* i *woman* - pokušajte ih ukloniti da vidite što će se dogoditi.

Kako bismo pronašli najbliži vektor, koristimo TensorFlow mehanizam za izračunavanje vektora udaljenosti između našeg vektora i svih vektora u rječniku, a zatim pronalazimo indeks minimalne riječi koristeći `argmin`.


Iako Word2Vec izgleda kao izvrstan način za izražavanje semantike riječi, ima mnoge nedostatke, uključujući sljedeće:

* I CBoW i skip-gram modeli su **prediktivni ugrađeni vektori**, i uzimaju u obzir samo lokalni kontekst. Word2Vec ne koristi prednosti globalnog konteksta.
* Word2Vec ne uzima u obzir **morfologiju** riječi, tj. činjenicu da značenje riječi može ovisiti o različitim dijelovima riječi, poput korijena.

**FastText** pokušava prevladati drugo ograničenje i nadograđuje se na Word2Vec učenjem vektorskih prikaza za svaku riječ i n-grama znakova unutar svake riječi. Vrijednosti tih prikaza zatim se prosječe u jedan vektor pri svakom koraku treniranja. Iako ovo dodaje puno dodatnih izračuna tijekom prethodnog treniranja, omogućuje ugrađenim vektorima riječi da kodiraju informacije o podriječima.

Druga metoda, **GloVe**, koristi drugačiji pristup ugrađivanju riječi, temeljen na faktorizaciji matrice konteksta riječi. Prvo, gradi veliku matricu koja broji koliko se puta riječi pojavljuju u različitim kontekstima, a zatim pokušava predstaviti tu matricu u nižim dimenzijama na način koji minimizira gubitak rekonstrukcije.

Knjižnica gensim podržava te ugrađene vektore riječi, i možete eksperimentirati s njima mijenjajući kod za učitavanje modela iznad.


## Korištenje unaprijed naučenih ugradnji u Kerasu

Možemo prilagoditi gornji primjer kako bismo unaprijed popunili matricu u našem sloju za ugradnju semantičkim ugradnjama, poput Word2Vec-a. Vokabulari unaprijed naučene ugradnje i tekstnog korpusa vjerojatno se neće podudarati, pa moramo odabrati jedan. Ovdje istražujemo dvije moguće opcije: korištenje vokabulara tokenizatora i korištenje vokabulara iz Word2Vec ugradnji.

### Korištenje vokabulara tokenizatora

Kada koristimo vokabular tokenizatora, neki od riječi iz vokabulara imat će odgovarajuće Word2Vec ugradnje, dok će neke nedostajati. S obzirom na to da je veličina našeg vokabulara `vocab_size`, a duljina vektora Word2Vec ugradnje `embed_size`, sloj za ugradnju bit će predstavljen težinskom matricom oblika `vocab_size`$\times$`embed_size`. Ovu matricu ćemo popuniti prolaskom kroz vokabular:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Za riječi koje nisu prisutne u Word2Vec rječniku, možemo ih ili ostaviti kao nule, ili generirati nasumični vektor.

Sada možemo definirati sloj za ugradnju s unaprijed istreniranim težinama:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Napomena**: Primijetite da smo postavili `trainable=False` prilikom kreiranja `Embedding`, što znači da ne treniramo ponovno sloj Embedding. Ovo može uzrokovati blago smanjenje točnosti, ali ubrzava proces treniranja.

### Korištenje vokabulara za ugradnju

Jedan od problema s prethodnim pristupom je taj što se vokabulari korišteni u TextVectorization i Embedding razlikuju. Kako bismo prevladali ovaj problem, možemo koristiti jedno od sljedećih rješenja:
* Ponovno trenirati Word2Vec model na našem vokabularu.
* Učitati naš skup podataka koristeći vokabular iz unaprijed istreniranog Word2Vec modela. Vokabulari koji se koriste za učitavanje skupa podataka mogu se specificirati tijekom učitavanja.

Drugi pristup čini se jednostavnijim, pa ćemo ga implementirati. Prije svega, kreirat ćemo sloj `TextVectorization` sa specificiranim vokabularom, preuzetim iz Word2Vec ugradnji:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Knjižnica gensim word embeddings sadrži praktičnu funkciju, `get_keras_embeddings`, koja će automatski stvoriti odgovarajući Keras embeddings sloj za vas.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Jedan od razloga zašto ne vidimo veću točnost je taj što neke riječi iz našeg skupa podataka nedostaju u unaprijed uvježbanom GloVe rječniku, i stoga su u biti zanemarene. Kako bismo to prevladali, možemo uvježbati vlastite ugrađene vektore na temelju našeg skupa podataka.


## Kontekstualni ugrađeni prikazi

Jedno od ključnih ograničenja tradicionalnih unaprijed istreniranih prikaza ugrađivanja, poput Word2Vec-a, jest činjenica da, iako mogu uhvatiti neko značenje riječi, ne mogu razlikovati različita značenja. To može uzrokovati probleme u modelima koji se koriste kasnije.

Na primjer, riječ 'play' ima različita značenja u ove dvije rečenice:
- Otišao sam na **predstavu** u kazalištu.
- John želi **igrati** se sa svojim prijateljima.

Unaprijed istrenirana ugrađivanja o kojima smo govorili predstavljaju oba značenja riječi 'play' u istom ugrađivanju. Kako bismo prevladali ovo ograničenje, trebamo izgraditi ugrađivanja temeljena na **jezičnom modelu**, koji je istreniran na velikom korpusu teksta i *zna* kako se riječi mogu povezivati u različitim kontekstima. Rasprava o kontekstualnim ugrađivanjima izlazi izvan okvira ovog vodiča, ali ćemo im se vratiti kada budemo govorili o jezičnim modelima u sljedećoj jedinici.



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane stručnjaka. Ne preuzimamo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja mogu proizaći iz korištenja ovog prijevoda.
